In [1]:
import pandas as pd
import DataLoad
import LibUtils

jobName = 'T201204_TrainingSet'
x, y = DataLoad.GetXY2(jobName)

print(x.columns, y.columns)

Index(['ElapsedMth', 'ContAge', 'Sex', 'Prem', 'PremYr', 'AccumPrem',
       'ContAmt', 'StartAgeOfSomething', 'SumFund', 'FundVal01', 'FundVal02',
       'FundVal03', 'FundAllo01', 'FundAllo02', 'FundAllo03', 'IR01', 'IR02',
       'IR03', 'IR04', 'IR05', 'IR06', 'IR07', 'IR08', 'IR09', 'IR10', 'IR11',
       'IR12', 'IR13', 'IR14', 'prd_0', 'prd_1', 'prd_2', 'prd_3', 'prd_4',
       'prd_5', 'prd_6', 'prd_7', 'prd_8', 'prd_9', 'prd_10', 'prd_11',
       'prd_12', 'prd_13', 'prd_14', 'prd_15'],
      dtype='object') Index(['pv_fee', 'pv_claim'], dtype='object')


In [37]:
from sklearn.model_selection import train_test_split

xVal, xTrain, yVal, yTrain = train_test_split(x, y, test_size=0.8)

In [35]:
import pandas as pd
from tensorflow import keras

#from keras import optimizers
#from keras import layers

model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=[len(x.columns)]),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)
    ])

opt = keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(opt, loss='mse')
#model.compile(loss='mse', optimizer=tf.keras.optimizers.RMSprop(0.001), metrics=['mae', 'mse'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                2944      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 5,057
Trainable params: 5,057
Non-trainable params: 0
_________________________________________________________________


In [55]:
yName = 'pv_claim'

early_stop = keras.callbacks.EarlyStopping(monitor='loss', patience=10)

model.fit(xTrain, yTrain[yName], epochs=1000, verbose=1, callbacks=[early_stop])

Epoch 1/1000
14000/14000 [==============================] - 16s 1ms/step - loss: 90521.7656
Epoch 2/1000
14000/14000 [==============================] - 16s 1ms/step - loss: 89133.9219
Epoch 3/1000
14000/14000 [==============================] - 16s 1ms/step - loss: 88491.1094
Epoch 4/1000
14000/14000 [==============================] - 16s 1ms/step - loss: 87967.7891
Epoch 5/1000
14000/14000 [==============================] - 16s 1ms/step - loss: 86590.6016
Epoch 6/1000
14000/14000 [==============================] - 17s 1ms/step - loss: 85864.0703
Epoch 7/1000
14000/14000 [==============================] - 15s 1ms/step - loss: 84169.2188
Epoch 8/1000
14000/14000 [==============================] - 15s 1ms/step - loss: 82956.2344
Epoch 9/1000
14000/14000 [==============================] - 16s 1ms/step - loss: 82747.2891
Epoch 10/1000
14000/14000 [==============================] - 15s 1ms/step - loss: 81021.0078
Epoch 11/1000
14000/14000 [==============================] - 15s 1ms/step - los

In [56]:
# 예측, 인덱스 재부여
yVal_predicted = pd.DataFrame(model.predict(xVal))
yVal_predicted.index = xVal.index

yTrain_predicted = pd.DataFrame(model.predict(xTrain))
yTrain_predicted.index = xTrain.index

In [57]:
scores_val = LibUtils.GetScores(yVal[yName], yVal_predicted[0])
scores_train = LibUtils.GetScores(yTrain[yName], yTrain_predicted[0])

print(scores_val)
print(scores_train)

[81.782593934992, 0.2820499663565116, 0.904279767306319]
[82.36382353230579, 0.28109502212375187, 0.9029233488895411]
